In [2]:
# FOMC 'Speeches' (연설문) 전용 크롤러 (2000 ~ 현재)
#
# [2025-10-30 v4 수정]
# - 0개 수집 문제 해결 (debug HTML 파일 기반)
# - v3의 'news-item' 파싱 로직을 버리고,
#   'eventlist', 'eventlist__time', 'eventlist__event', 'news__speaker'
#   클래스를 사용하도록 파싱 로직 전면 수정.
# - 날짜 형식을 '%m/%d/%y' (2자리 연도) -> '%m/%d/%Y' (4자리 연도)로 수정.
#
# [동작 방식 (하이브리드)]
# 2000년부터 2025년까지 루프를 돌면서,
# 1. '.../{year}-speeches.htm' (2011-현재) 패턴을 먼저 시도
# 2. 404가 발생하면 '.../{year}speech.htm' (2000-2010) 패턴을 시도
#
# [중요] 실행 전 설정:
# 1. 터미널에서 `pip install requests beautifulsoup4 PyMuPDF tqdm`을 실행하여
#    라이브러리를 설치하세요.

import requests
from bs4 import BeautifulSoup
import fitz  # PyMuPDF
import json
import time
import os
import datetime
from urllib.parse import urljoin
from tqdm import tqdm
import re

# --- 1. 설정 (Configuration) ---
BASE_URL = "https://www.federalreserve.gov"
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}
URL_TEMPLATE_NEW = "https://www.federalreserve.gov/newsevents/speech/{year}-speeches.htm"
URL_TEMPLATE_OLD = "https://www.federalreserve.gov/newsevents/speech/{year}speech.htm"

DOWNLOADS_DIR = os.path.join(os.path.expanduser('~'), 'Downloads', 'statements')
OUTPUT_FILE = os.path.join(DOWNLOADS_DIR, "fomc_speeches_2000-present.jsonl")
START_YEAR = 2000 
END_YEAR = datetime.datetime.now().year

# --- 2. 헬퍼 함수 (Helper Functions) ---

def save_to_jsonl(data, outfile):
    """데이터를 JSON Lines (.jsonl) 파일에 추가합니다."""
    try:
        output_dir = os.path.dirname(outfile)
        if output_dir and not os.path.exists(output_dir):
            os.makedirs(output_dir)
            
        with open(outfile, 'a', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False)
            f.write('\n')
    except IOError as e:
        print(f"  [오류] 파일 쓰기 실패: {e}")

def get_soup(url):
    """지정된 URL의 BeautifulSoup 객체를 반환합니다."""
    try:
        response = requests.get(url, headers=HEADERS)
        if response.status_code == 404:
            return None # 404는 흔한 경우이므로 조용히 None 반환
        response.raise_for_status()
        response.encoding = response.apparent_encoding 
        return BeautifulSoup(response.text, 'html.parser')
    except requests.exceptions.RequestException as e:
        print(f"  [오류] URL에 접근할 수 없습니다: {url} ({e})")
        return None

def clean_text(text):
    """불필요한 공백과 줄바꿈 문자를 정제합니다."""
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

def extract_text_from_link(link_url):
    """HTML 또는 PDF 링크에서 텍스트를 깨끗하게 추출합니다."""
    text = ""
    try:
        full_url = urljoin(BASE_URL, link_url)
        
        response = requests.get(full_url, headers=HEADERS, timeout=10)
        response.raise_for_status()

        if ".pdf" in full_url.lower():
            with fitz.open(stream=response.content, filetype="pdf") as doc:
                for page in doc:
                    text += page.get_text() + "\n"
        else:
            soup = BeautifulSoup(response.text, 'html.parser')
            content = soup.find('div', id='article')
            if not content:
                content_paragraphs = soup.find_all('p')
                if content_paragraphs:
                    text = "\n".join([p.get_text() for p in content_paragraphs])
                else:
                    content = soup.find('body')
            
            if content and not text:
                for script_or_style in content(['script', 'style']):
                    script_or_style.decompose()
                text = content.get_text(separator="\n", strip=True)
            elif not text:
                text = soup.get_text(separator="\n", strip=True)

    except Exception as e:
        print(f"  [오류] 링크에서 텍스트 추출 실패: {full_url} ({e})")
        
    time.sleep(0.5) 
    return clean_text(text)

# --- 3. 메인 스크래핑 함수 (v4: 'eventlist' 구조 파싱) ---
def scrape_all_speeches(output_file):
    print(f"--- FOMC 'Speeches' (연설문) 스크래핑 시작 (v4) ---")
    print(f"대상 기간: {START_YEAR}년 ~ {END_YEAR}년")
    
    scraped_count = 0
    total_items_processed = 0
    
    for year in tqdm(range(END_YEAR, START_YEAR - 1, -1), desc="연도별 수집"):
        
        url = URL_TEMPLATE_NEW.format(year=year)
        soup = get_soup(url)
        
        if soup is None:
            time.sleep(0.5)
            url = URL_TEMPLATE_OLD.format(year=year)
            soup = get_soup(url)

        if soup is None:
            print(f"  [정보] {year}년 - 두 URL 패턴 모두 찾지 못함. 건너뜁니다.")
            continue
            
        # [v4 수정] 'eventlist' div를 찾음
        eventlist_container = soup.find('div', class_='eventlist')
        if not eventlist_container:
            eventlist_container = soup.find('div', id='article') # Fallback
            if not eventlist_container:
                print(f"  [경고] {year}년 - 'eventlist' 또는 'article' div를 찾지 못함.")
                continue

        # [v4 수정] eventlist 안의 모든 'row'가 개별 연설 항목임
        speech_items = eventlist_container.find_all('div', class_='row')
            
        for item in speech_items:
            # [v4 수정] 'time' 태그와 'eventlist__event' 클래스로 검색
            date_element = item.find('time')
            event_element = item.find('div', class_='eventlist__event')
            
            if not date_element or not event_element:
                continue # 날짜나 이벤트 내용이 없는 row (예: 헤더, 공백)는 건너뜀

            try:
                date_text = date_element.get_text(strip=True)
                # [v4 수정] '%m/%d/%Y' (4자리 연도)로 파싱 (예: 12/3/2024)
                meeting_date = datetime.datetime.strptime(date_text, '%m/%d/%Y')
                
                # 수집 중인 연도와 날짜가 일치하는지 확인
                if meeting_date.year != year:
                    continue 

                # [v4 수정] 'eventlist__event' 안의 첫 <p><a> 태그
                title_link = event_element.find('p').find('a')
                if not title_link:
                    continue
                    
                # [v4 수정] 'news__speaker' 클래스로 연설자 검색
                speaker_element = event_element.find('p', class_='news__speaker')

                link_url = title_link.get('href')
                title = clean_text(title_link.get_text())
                speaker = clean_text(speaker_element.get_text()) if speaker_element else "Unknown"

                total_items_processed += 1
                text = extract_text_from_link(link_url)
                
                if text:
                    data = {
                        "date": meeting_date.strftime('%Y-%m-%d'), 
                        "year": meeting_date.year,
                        "source_type": "FOMC Speech",
                        "speaker": speaker,
                        "title": title,
                        "url": urljoin(BASE_URL, link_url),
                        "text": text
                    }
                    save_to_jsonl(data, output_file)
                    scraped_count += 1
            
            except Exception as e:
                # print(f"  [정보] {year}년 - 연설문 항목 파싱 실패: {e}")
                pass
                        
    print(f"\n--- (2000년 ~ {END_YEAR}년) 총 {total_items_processed}개 항목 중 {scraped_count}개의 'Speeches' 문서를 수집했습니다.")

# --- 4. 메인 실행 ---
if __name__ == "__main__":
    if not os.path.exists(DOWNLOADS_DIR):
        os.makedirs(DOWNLOADS_DIR)
        print(f"'{DOWNLOADS_DIR}' 폴더를 생성했습니다.")

    if os.path.exists(OUTPUT_FILE):
        os.remove(OUTPUT_FILE)
        print(f"기존 파일 '{OUTPUT_FILE}'을 삭제했습니다. 새로 수집합니다.")
    
    scrape_all_speeches(OUTPUT_FILE)
    
    print(f"\n--- 모든 'Speeches' 작업 완료 ---")
    print(f"최종 데이터가 '{OUTPUT_FILE}' 파일에 저장되었습니다.")

--- FOMC 'Speeches' (연설문) 스크래핑 시작 (v4) ---
대상 기간: 2000년 ~ 2025년


연도별 수집:  46%|████▌     | 12/26 [15:21<13:56, 59.76s/it]

  [오류] 링크에서 텍스트 추출 실패: https://www.federalreserve.gov/newsevents/speech/bernanke20131119a.htm (HTTPSConnectionPool(host='www.federalreserve.gov', port=443): Read timed out. (read timeout=10))


연도별 수집:  81%|████████  | 21/26 [25:27<05:00, 60.14s/it]

  [경고] 2005년 - 'eventlist' 또는 'article' div를 찾지 못함.


...

  [경고] 2002년 - 'eventlist' 또는 'article' div를 찾지 못함.


연도별 수집:  96%|█████████▌| 25/26 [25:36<00:15, 15.89s/it]

  [경고] 2001년 - 'eventlist' 또는 'article' div를 찾지 못함.


연도별 수집: 100%|██████████| 26/26 [25:38<00:00, 59.18s/it]

  [경고] 2000년 - 'eventlist' 또는 'article' div를 찾지 못함.

--- (2000년 ~ 2025년) 총 1268개 항목 중 1267개의 'Speeches' 문서를 수집했습니다.

--- 모든 'Speeches' 작업 완료 ---
최종 데이터가 'C:\Users\82109\Downloads\statements\fomc_speeches_2000-present.jsonl' 파일에 저장되었습니다.
